In [1]:
!pip install torch
!pip install transformers
!pip install json

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [25]:
import json
import torch
from transformers import BertTokenizer, BertForTokenClassification, AdamW

In [33]:
import json
import torch
import random 
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizer, BertForTokenClassification, AdamW

# Load the labeled dataset
with open("filtered_utterances_ft_data.json", "r") as f:
    labeled_data = json.load(f)

# Load the pragmatic marker lists
with open("hedging_markers.json", "r") as f:
    hedging_markers = json.load(f)

with open("authority_markers.json", "r") as f:
    authority_markers = json.load(f)

# Prepare the data for training
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
labels = ["none", "hedge", "authority"]
label_map = {label: i for i, label in enumerate(labels)}

# Split the dataset into train and eval subsets
random.shuffle(labeled_data)
train_size = int(0.8 * len(labeled_data))
train_data = labeled_data[:train_size]
eval_data = labeled_data[train_size:]

max_length = 512  # Set the maximum sequence length to 512

train_dataset = []
for example in train_data:
    statement = example["statement"]
    encoding = tokenizer.encode_plus(
        statement,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )
    input_ids = encoding["input_ids"].squeeze()
    attention_mask = encoding["attention_mask"].squeeze()

    label_ids = [label_map["none"]] * max_length  # Initialize label_ids with "none"
    for term, label in example["matched_terms"].items():
        if label != "none":
            term_tokens = tokenizer.tokenize(term)
            term_indices = [i for i, token_id in enumerate(input_ids) if tokenizer.decode([token_id]).strip().lower() in term_tokens]
            for index in term_indices:
                label_ids[index] = label_map[label]

    train_dataset.append({
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": torch.tensor(label_ids),
    })
    
eval_dataset = []
for example in eval_data:
    statement = example["statement"]
    encoding = tokenizer.encode_plus(
        statement,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )
    input_ids = encoding["input_ids"].squeeze()
    attention_mask = encoding["attention_mask"].squeeze()

    label_ids = [label_map["none"]] * max_length  # Initialize label_ids with "none"
    for term, label in example["matched_terms"].items():
        if label != "none":
            term_tokens = tokenizer.tokenize(term)
            term_indices = [i for i, token_id in enumerate(input_ids) if tokenizer.decode([token_id]).strip().lower() in term_tokens]
            for index in term_indices:
                label_ids[index] = label_map[label]

    eval_dataset.append({
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": torch.tensor(label_ids),
    })    
    
    

# Set up the model and optimizer
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(labels))
optimizer = AdamW(model.parameters(), lr=2e-5)

# Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

batch_size = 8
epochs = 3

for epoch in range(epochs):
    for i in range(0, len(dataset), batch_size):
        batch = dataset[i:i+batch_size]
        
        input_ids = torch.stack([example["input_ids"] for example in batch]).to(device)
        attention_mask = torch.stack([example["attention_mask"] for example in batch]).to(device)
        labels = torch.stack([example["labels"] for example in batch]).to(device)

        model.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the fine-tuned model
model.save_pretrained("pragmatic_markers_model")

# Evaluate the model
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for example in eval_dataset:
        input_ids = example["input_ids"].unsqueeze(0).to(device)
        attention_mask = example["attention_mask"].unsqueeze(0).to(device)
        labels = example["labels"].unsqueeze(0).to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        predictions = outputs.logits.argmax(dim=-1).squeeze().tolist()

        correct_predictions += sum(p == l for p, l in zip(predictions, labels.squeeze().tolist()) if l != -100)
        total_predictions += len([p for p, l in zip(predictions, labels.squeeze().tolist()) if l != -100])

accuracy = correct_predictions / total_predictions
print(f"Evaluation Accuracy: {accuracy:.4f}")

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.76ea01b4b85ac16e2cec55c398cba7a943d89ab21dfdd973f6630a152e4b9aed
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7

Evaluation Accuracy: 0.9990
